## Focus: Integrity, Timezone Detection, and Raw Extraction.

### 1. Imports & Initialization

In [ ]:
# Cell 1: Imports & Init
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timezone
import pytz
import os

# Initialize MT5
if not mt5.initialize():
    print(f"❌ MT5 Initialize Failed: {mt5.last_error()}")
    quit()
else:
    print(f"✅ Connected to MT5. Account: {mt5.account_info().login}")

# Ensure Data Directory
DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)
RAW_FILE = os.path.join(DATA_DIR, "EURUSD_H1_Raw.parquet")

### 2: Timezone Detective (The Fix)

In [ ]:
# Cell 2: Timezone Detective (The Fix)
# We need to know what time the broker is on relative to UTC
symbol_info = mt5.symbol_info("EURUSD")
broker_time = datetime.fromtimestamp(mt5.symbol_info_tick("EURUSD").time)
local_utc = datetime.now(timezone.utc).replace(tzinfo=None)

# Calculate rough offset (Broker Time - UTC)
diff_hours = round((broker_time - local_utc).total_seconds() / 3600)

print(f"🌍 UTC Time:    {local_utc}")
print(f"🏦 Broker Time: {broker_time}")
print(f"⏱️ Detected Offset: UTC{'+' if diff_hours >= 0 else ''}{diff_hours}")

# We will store this offset metadata or just normalize everything to UTC immediately

### 3: Extraction

In [ ]:
# Cell 3: Extraction
SYMBOL = "EURUSD"
TIMEFRAME = mt5.TIMEFRAME_H1
utc_from = datetime(2020, 1, 1, tzinfo=timezone.utc)
utc_to = datetime.now(timezone.utc)

print(f"⏳ Extracting {SYMBOL} H1 from {utc_from.date()} to {utc_to.date()}...")
rates = mt5.copy_rates_range(SYMBOL, TIMEFRAME, utc_from, utc_to)

if rates is None:
    print("❌ No data found!")
else:
    df = pd.DataFrame(rates)
    # MT5 timestamps are usually in BROKER TIME (seconds epoch).
    # We convert to datetime. 
    # CRITICAL: We explicitly set the timezone to the Detected Offset from Cell 2
    # For FBS, it's usually UTC+2 or UTC+3. Let's assume the broker returns standard timestamps.
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
    # We assume the timestamp provided by copy_rates_range is in the Broker's Timezone.
    # To standardize, we will treat the index as "Broker Local Time" for now.
    df.set_index('time', inplace=True)
    
    # Keep relevant columns
    df = df[['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume']]
    
    print(f"✅ Fetched {len(df)} rows.")
    print(df.tail(3))

### 4: Save to Parquet

In [ ]:
# Cell 4: Save
df.to_parquet(RAW_FILE)
print(f"💾 Saved Raw Data to {RAW_FILE}")
mt5.shutdown()